# Inspect Tree Training Data

Inspect and visualize data loading and pre-processing code.

In [3]:
import concurrent.futures
import itertools
import json
import logging
import math
import os
import random
import re
import sys
import time

import imgaug
import matplotlib
import matplotlib.lines as lines
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from imgaug import augmenters as iaa
from matplotlib.patches import Polygon

# Root directory of the project
MRCNN_DIR = os.path.abspath("../Mask_RCNN/")
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(MRCNN_DIR)
from mrcnn import model as modellib
from mrcnn import utils, visualize
from mrcnn.model import log
from mrcnn.visualize import display_images

import tree

%matplotlib inline
%load_ext autoreload
%autoreload 2

2022-10-11 22:34:32.411921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Configurations

In [ ]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, "data/mrcnn")


# Use configuation from tree.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(tree.TreeConfig):
    IMAGE_RESIZE_MODE = "none"
    USE_MINI_MASK = False


config = NoResizeConfig()

## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.

    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size * cols, size * rows))
    return ax

## Dataset

Organize the data to better fit with Mask RCNN's assumptions about data structure

In [5]:
organize_data = True

if organize_data:
    import shutil

    DATASET_DIR = os.path.join(ROOT_DIR, "data")
    rgb_dir = os.path.join(DATASET_DIR, "rgb_imp")
    label_dir = os.path.join(DATASET_DIR, "label_imp")

    image_ids = os.listdir(rgb_dir)
    image_ids = [x.split(".")[0] for x in image_ids]
    label_ids = os.listdir(label_dir)
    label_ids = [x.split(".")[0] for x in label_ids]

    image_ids.sort()
    label_ids.sort()

    for i, img in enumerate(image_ids):
        # random_bits = random.getrandbits(128)
        # hash1 = "%032x" % random_bits
        os.makedirs(f"{DATASET_DIR}/mrcnn_imp/{img}/image")
        os.makedirs(f"{DATASET_DIR}/mrcnn_imp/{img}/mask")
        shutil.copyfile(
            f"{DATASET_DIR}/rgb/{img}.tif", f"{DATASET_DIR}/mrcnn_imp/{img}/image/{img}.tif"
        )
        shutil.copyfile(
            f"{label_dir}/{label_ids[i]}.tif",
            f"{DATASET_DIR}/mrcnn_imp/{img}/mask/{label_ids[i]}.tif",
        )

FileExistsError: [Errno 17] File exists: '/Users/lusk/projects/uni-potsdam/so22/dap05/proj/tree/data/mrcnn_imp/393_5823_RGB_2020_01_00/image'

In [ ]:
# Load dataset
dataset = tree.TreeDataset()

dataset.load_tree(DATASET_DIR, subset="hand", split=0.1)

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info["name"]))

In [ ]:
dataset.image_info[0]

In [ ]:
len(dataset.image_ids)

## Display Samples

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=1)

In [ ]:
# Example of loading a specific image by its source ID
source_id = "393_5823_RGB_2020_01_03"

# Map source ID to Dataset image_id
# Notice the tree prefix: it's the name given to the dataset in TreeDataset
image_id = dataset.image_from_source_map["tree.{}".format(source_id)]

# Load and display
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset, config, image_id
)
log("molded_image", image)
log("mask", mask)
visualize.display_instances(
    image, bbox, mask, class_ids, dataset.class_names, show_bbox=False
)

## Dataset Stats

Loop through all images in the dataset and collect aggregate stats.

In [ ]:
def image_stats(image_id):
    """Returns a dict of stats for one image."""
    image = dataset.load_image(image_id)
    mask, _ = dataset.load_mask(image_id)
    bbox = utils.extract_bboxes(mask)
    # Sanity check
    assert mask.shape[:2] == image.shape[:2]
    # Return stats dict
    return {
        "id": image_id,
        "shape": list(image.shape),
        "bbox": [
            [b[2] - b[0], b[3] - b[1]]
            for b in bbox
            # Uncomment to exclude trees with 1 pixel width
            # or height (often on edges)
            # if b[2] - b[0] > 1 and b[3] - b[1] > 1
        ],
        "color": np.mean(image, axis=(0, 1)),
    }


# Loop through the dataset and compute stats over multiple threads
# This might take a few minutes
t_start = time.time()
with concurrent.futures.ThreadPoolExecutor() as e:
    stats = list(e.map(image_stats, dataset.image_ids))
t_total = time.time() - t_start
print("Total time: {:.1f} seconds".format(t_total))

### Image Size Stats

In [ ]:
# Image stats
image_shape = np.array([s["shape"] for s in stats])
image_color = np.array([s["color"] for s in stats])
print("Image Count: ", image_shape.shape[0])
print(
    "Height  mean: {:.2f}  median: {:.2f}  min: {:.2f}  max: {:.2f}".format(
        np.mean(image_shape[:, 0]),
        np.median(image_shape[:, 0]),
        np.min(image_shape[:, 0]),
        np.max(image_shape[:, 0]),
    )
)
print(
    "Width   mean: {:.2f}  median: {:.2f}  min: {:.2f}  max: {:.2f}".format(
        np.mean(image_shape[:, 1]),
        np.median(image_shape[:, 1]),
        np.min(image_shape[:, 1]),
        np.max(image_shape[:, 1]),
    )
)
print("Color   mean (RGB): {:.2f} {:.2f} {:.2f}".format(*np.mean(image_color, axis=0)))

# Histograms
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].set_title("Height")
_ = ax[0].hist(image_shape[:, 0], bins=20)
ax[1].set_title("Width")
_ = ax[1].hist(image_shape[:, 1], bins=20)
ax[2].set_title("Height & Width")
_ = ax[2].hist2d(image_shape[:, 1], image_shape[:, 0], bins=10, cmap="Blues")

### Trees per Image Stats

In [ ]:
image_area = 512**2

print("Tree/Image")
fig, ax = plt.subplots(1)
area_threshold = 0

tree_per_image = np.array(
    [
        len(s["bbox"])
        for s in stats
        if area_threshold < (s["shape"][0] * s["shape"][1]) <= image_area
    ]
)
area_threshold = image_area
if len(tree_per_image) == 0:
    print("Image area <= {:4}**2: None".format(np.sqrt(image_area)))

print(
    "Image area = {:4.0f}**2:  mean: {:.1f}  median: {:.1f}  min: {:.1f}  max: {:.1f}".format(
        np.sqrt(image_area),
        tree_per_image.mean(),
        np.median(tree_per_image),
        tree_per_image.min(),
        tree_per_image.max(),
    )
)
ax.set_title("Image Area = {:4}**2".format(np.sqrt(image_area)))
_ = ax.hist(tree_per_image, bins=10)

### Tree Size Stats

In [ ]:
# Trees size stats
fig, ax = plt.subplots(1)
area_threshold = 0

tree_shape = np.array(
    [
        b
        for s in stats
        if area_threshold < (s["shape"][0] * s["shape"][1]) <= image_area
        for b in s["bbox"]
    ]
)
tree_area = tree_shape[:, 0] * tree_shape[:, 1]
area_threshold = image_area

print("\nImage Area = {:.0f}**2".format(np.sqrt(image_area)))
print("  Total Trees: ", tree_shape.shape[0])
print(
    "  Tree Height. mean: {:.2f}  median: {:.2f}  min: {:.2f}  max: {:.2f}".format(
        np.mean(tree_shape[:, 0]),
        np.median(tree_shape[:, 0]),
        np.min(tree_shape[:, 0]),
        np.max(tree_shape[:, 0]),
    )
)
print(
    "  Tree Width.  mean: {:.2f}  median: {:.2f}  min: {:.2f}  max: {:.2f}".format(
        np.mean(tree_shape[:, 1]),
        np.median(tree_shape[:, 1]),
        np.min(tree_shape[:, 1]),
        np.max(tree_shape[:, 1]),
    )
)
print(
    "  Tree Area.   mean: {:.2f}  median: {:.2f}  min: {:.2f}  max: {:.2f}".format(
        np.mean(tree_area),
        np.median(tree_area),
        np.min(tree_area),
        np.max(tree_area),
    )
)

# Show 2D histogram
_ = ax.hist2d(tree_shape[:, 1], tree_shape[:, 0], bins=20, cmap="Blues")

In [ ]:
# Trees height/width ratio
tree_aspect_ratio = tree_shape[:, 0] / tree_shape[:, 1]
print(
    "Tree Aspect Ratio.  mean: {:.2f}  median: {:.2f}  min: {:.2f}  max: {:.2f}".format(
        np.mean(tree_aspect_ratio),
        np.median(tree_aspect_ratio),
        np.min(tree_aspect_ratio),
        np.max(tree_aspect_ratio),
    )
)
plt.figure(figsize=(15, 5))
_ = plt.hist(tree_aspect_ratio, bins=100, range=[0, 5])

## Image Augmentation

Test out different augmentation methods

In [ ]:
# List of augmentations
# http://imgaug.readthedocs.io/en/latest/source/augmenters.html
augmentation = iaa.Sometimes(
    0.9,
    [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Multiply((0.8, 1.2)),
        iaa.GaussianBlur(sigma=(0.0, 5.0)),
    ],
)

In [ ]:
# Load the image multiple times to show augmentations
limit = 4
ax = get_ax(rows=2, cols=limit // 2)
for i in range(limit):
    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
        dataset,
        config,
        image_id,
        augmentation=augmentation,
    )
    visualize.display_instances(
        image,
        bbox,
        mask,
        class_ids,
        dataset.class_names,
        ax=ax[i // 2, i % 2],
        show_mask=False,
        show_bbox=False,
    )

## Image Crops

Aerial images tend to be large, but trees are small. So it's more efficient to train on random crops from large images. This is handled by `config.IMAGE_RESIZE_MODE = "crop"`.



In [ ]:
class RandomCropConfig(tree.TreeConfig):
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128


crop_config = RandomCropConfig()

In [ ]:
# Load the image multiple times to show different crops
limit = 4
image_id = np.random.choice(dataset.image_ids, 1)[0]
ax = get_ax(rows=2, cols=limit // 2)
for i in range(limit):
    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
        dataset, crop_config, image_id
    )
    visualize.display_instances(
        image,
        bbox,
        mask,
        class_ids,
        dataset.class_names,
        ax=ax[i // 2, i % 2],
        show_mask=False,
        show_bbox=False,
    )

## Mini Masks

Instance binary masks can get large when training with high resolution images. For example, if training with 1024x1024 image then the mask of a single instance requires 1MB of memory (Numpy uses bytes for boolean values). If an image has 100 instances then that's 100MB for the masks alone. 

To improve training speed, we optimize masks:
* We store mask pixels that are inside the object bounding box, rather than a mask of the full image. Most objects are small compared to the image size, so we save space by not storing a lot of zeros around the object.
* We resize the mask to a smaller size (e.g. 56x56). For objects that are larger than the selected size we lose a bit of accuracy. But most object annotations are not very accuracy to begin with, so this loss is negligable for most practical purposes. Thie size of the mini_mask can be set in the config class.

To visualize the effect of mask resizing, and to verify the code correctness, we visualize some examples.

In [ ]:
# Load random image and mask.
image_id = np.random.choice(dataset.image_ids, 1)[0]
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
original_shape = image.shape
# Resize
image, window, scale, padding, _ = utils.resize_image(
    image,
    min_dim=config.IMAGE_MIN_DIM,
    max_dim=config.IMAGE_MAX_DIM,
    mode=config.IMAGE_RESIZE_MODE,
)
mask = utils.resize_mask(mask, scale, padding)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id: ", image_id, dataset.image_reference(image_id))
print("Original shape: ", original_shape)
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [ ]:
image_id = np.random.choice(dataset.image_ids, 1)[0]
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset, config, image_id
)

log("image", image)
log("image_meta", image_meta)
log("class_ids", class_ids)
log("bbox", bbox)
log("mask", mask)

display_images([image] + [mask[:, :, i] for i in range(min(mask.shape[-1], 7))])

In [ ]:
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [ ]:
class MiniMaskConfig(RandomCropConfig):
    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask


mm_config = MiniMaskConfig()

# Add augmentation and mask resizing.
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset, mm_config, image_id, augmentation=augmentation
)
log("mask", mask)
display_images([image] + [mask[:, :, i] for i in range(min(mask.shape[-1], 7))])

In [ ]:
mask = utils.expand_mask(bbox, mask, image.shape)
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

## Anchors

For an FPN network, the anchors must be ordered in a way that makes it easy to match anchors to the output of the convolution layers that predict anchor scores and shifts. 
* Sort by pyramid level first. All anchors of the first level, then all of the second and so on. This makes it easier to separate anchors by level.
* Within each level, sort anchors by feature map processing sequence. Typically, a convolution layer processes a feature map starting from top-left and moving right row by row. 
* For each feature map cell, pick any sorting order for the anchors of different ratios. Here we match the order of ratios passed to the function.

In [ ]:
## Visualize anchors of one cell at the center of the feature map

# Load and display random image
image_id = np.random.choice(dataset.image_ids, 1)[0]
image, image_meta, _, _, _ = modellib.load_image_gt(dataset, crop_config, image_id)

# Generate Anchors
backbone_shapes = modellib.compute_backbone_shapes(config, image.shape)
anchors = utils.generate_pyramid_anchors(
    config.RPN_ANCHOR_SCALES,
    config.RPN_ANCHOR_RATIOS,
    backbone_shapes,
    config.BACKBONE_STRIDES,
    config.RPN_ANCHOR_STRIDE,
)

# Print summary of anchors
num_levels = len(backbone_shapes)
anchors_per_cell = len(config.RPN_ANCHOR_RATIOS)
print("Count: ", anchors.shape[0])
print("Scales: ", config.RPN_ANCHOR_SCALES)
print("ratios: ", config.RPN_ANCHOR_RATIOS)
print("Anchors per Cell: ", anchors_per_cell)
print("Levels: ", num_levels)
anchors_per_level = []
for l in range(num_levels):
    num_cells = backbone_shapes[l][0] * backbone_shapes[l][1]
    anchors_per_level.append(
        anchors_per_cell * num_cells // config.RPN_ANCHOR_STRIDE**2
    )
    print("Anchors in Level {}: {}".format(l, anchors_per_level[l]))

# Display
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.imshow(image)
levels = len(backbone_shapes)

for level in range(levels):
    colors = visualize.random_colors(levels)
    # Compute the index of the anchors at the center of the image
    level_start = sum(anchors_per_level[:level])  # sum of anchors of previous levels
    level_anchors = anchors[level_start : level_start + anchors_per_level[level]]
    print(
        "Level {}. Anchors: {:6}  Feature map Shape: {}".format(
            level, level_anchors.shape[0], backbone_shapes[level]
        )
    )
    center_cell = backbone_shapes[level] // 2
    center_cell_index = center_cell[0] * backbone_shapes[level][1] + center_cell[1]
    level_center = center_cell_index * anchors_per_cell
    center_anchor = anchors_per_cell * (
        (center_cell[0] * backbone_shapes[level][1] / config.RPN_ANCHOR_STRIDE**2)
        + center_cell[1] / config.RPN_ANCHOR_STRIDE
    )
    level_center = int(center_anchor)

    # Draw anchors. Brightness show the order in the array, dark to bright.
    for i, rect in enumerate(
        level_anchors[level_center : level_center + anchors_per_cell]
    ):
        y1, x1, y2, x2 = rect
        p = patches.Rectangle(
            (x1, y1),
            x2 - x1,
            y2 - y1,
            linewidth=2,
            facecolor="none",
            edgecolor=(i + 1) * np.array(colors[level]) / anchors_per_cell,
        )
        ax.add_patch(p)

## Data Generator

In [ ]:
random_rois = 2000

g = modellib.DataGenerator(
    dataset,
    mm_config,
    augmentation=augmentation,
    random_rois=random_rois,
    detection_targets=True,
)

In [ ]:
# # Create data generator
# random_rois = 2000
# g = modellib.data_generator(
#     dataset,
#     crop_config,
#     shuffle=True,
#     random_rois=random_rois,
#     batch_size=4,
#     detection_targets=True,
# )

In [ ]:
# Uncomment to run the generator through a lot of images
# to catch rare errors

# Ignore the imgaug warnings
import warnings

warnings.filterwarnings("ignore")

for i in range(1000):
    print(i)
    _, _ = g[i]

In [ ]:
# Get Next Image
if random_rois:
    [
        normalized_images,
        image_meta,
        rpn_match,
        rpn_bbox,
        gt_class_ids,
        gt_boxes,
        gt_masks,
        rpn_rois,
        rois,
    ], [mrcnn_class_ids, mrcnn_bbox, mrcnn_mask] = next(iter(g))

    log("rois", rois)
    log("mrcnn_class_ids", mrcnn_class_ids)
    log("mrcnn_bbox", mrcnn_bbox)
    log("mrcnn_mask", mrcnn_mask)
else:
    [normalized_images, image_meta, rpn_match, rpn_bbox, gt_boxes, gt_masks], _ = next(
        iter(g)
    )

log("gt_class_ids", gt_class_ids)
log("gt_boxes", gt_boxes)
log("gt_masks", gt_masks)
log(
    "rpn_match",
    rpn_match,
)
log("rpn_bbox", rpn_bbox)
image_id = modellib.parse_image_meta(image_meta)["image_id"][0]
print("image_id: ", image_id, dataset.image_reference(image_id))

# Remove the last dim in mrcnn_class_ids. It's only added
# to satisfy Keras restriction on target shape.
mrcnn_class_ids = mrcnn_class_ids[:, :, 0]

In [ ]:
b = 0

# Restore original image (reverse normalization)
sample_image = modellib.unmold_image(normalized_images[b], config)

# Compute anchor shifts.
indices = np.where(rpn_match[b] == 1)[0]
refined_anchors = utils.apply_box_deltas(
    anchors[indices], rpn_bbox[b, : len(indices)] * config.RPN_BBOX_STD_DEV
)
log("anchors", anchors)
log("refined_anchors", refined_anchors)

# Get list of positive anchors
positive_anchor_ids = np.where(rpn_match[b] == 1)[0]
print("Positive anchors: {}".format(len(positive_anchor_ids)))
negative_anchor_ids = np.where(rpn_match[b] == -1)[0]
print("Negative anchors: {}".format(len(negative_anchor_ids)))
neutral_anchor_ids = np.where(rpn_match[b] == 0)[0]
print("Neutral anchors: {}".format(len(neutral_anchor_ids)))

# ROI breakdown by class
for c, n in zip(dataset.class_names, np.bincount(mrcnn_class_ids[b].flatten())):
    if n:
        print("{:23}: {}".format(c[:20], n))

# Show positive anchors
fig, ax = plt.subplots(1, figsize=(16, 16))
visualize.draw_boxes(
    sample_image,
    boxes=anchors[positive_anchor_ids],
    refined_boxes=refined_anchors,
    ax=ax,
)

In [ ]:
# Show negative anchors
visualize.draw_boxes(sample_image, boxes=anchors[negative_anchor_ids])

In [ ]:
# Show neutral anchors. They don't contribute to training.
visualize.draw_boxes(
    sample_image, boxes=anchors[np.random.choice(neutral_anchor_ids, 100)]
)

## ROIs

Typically, the RPN network generates region proposals (a.k.a. Regions of Interest, or ROIs). The data generator has the ability to generate proposals as well for illustration and testing purposes. These are controlled by the `random_rois` parameter.

In [ ]:
if random_rois:
    # Class aware bboxes
    bbox_specific = mrcnn_bbox[b, np.arange(mrcnn_bbox.shape[1]), mrcnn_class_ids[b], :]

    # Refined ROIs
    refined_rois = utils.apply_box_deltas(
        rois[b].astype(np.float32), bbox_specific[:, :4] * config.BBOX_STD_DEV
    )

    # Class aware masks
    mask_specific = mrcnn_mask[
        b, np.arange(mrcnn_mask.shape[1]), :, :, mrcnn_class_ids[b]
    ]

    visualize.draw_rois(
        sample_image,
        rois[b],
        refined_rois,
        mask_specific,
        mrcnn_class_ids[b],
        dataset.class_names,
    )

    # Any repeated ROIs?
    rows = np.ascontiguousarray(rois[b]).view(
        np.dtype((np.void, rois.dtype.itemsize * rois.shape[-1]))
    )
    _, idx = np.unique(rows, return_index=True)
    print("Unique ROIs: {} out of {}".format(len(idx), rois.shape[1]))

In [ ]:
if random_rois:
    # Dispalay ROIs and corresponding masks and bounding boxes
    ids = random.sample(range(rois.shape[1]), 8)

    images = []
    titles = []
    for i in ids:
        image = visualize.draw_box(
            sample_image.copy(), rois[b, i, :4].astype(np.int32), [255, 0, 0]
        )
        image = visualize.draw_box(image, refined_rois[i].astype(np.int64), [0, 255, 0])
        images.append(image)
        titles.append("ROI {}".format(i))
        images.append(mask_specific[i] * 255)
        titles.append(dataset.class_names[mrcnn_class_ids[b, i]][:20])

    display_images(images, titles, cols=4, cmap="Blues", interpolation="none")

TODO: For some reason temp_g is returning the same exact ids for each batch...

In [ ]:
# Check ratio of positive ROIs in a set of images.
if random_rois:
    limit = 10
    temp_g = modellib.DataGenerator(
        dataset,
        mm_config,
        shuffle=True,
        augmentation=augmentation,
        random_rois=10000,
        detection_targets=True,
    )
    total = 0
    inputses = []
    idses = []
    for i in range(limit):
        inputs, [ids, _, _] = temp_g[i]
        inputses.append(inputs)
        idses.append(ids)
        if i > 0 and (np.max(idses[i] - idses[i - 1]) == 0):
            print("ids are the same")
        positive_rois = np.sum(ids[0] > 0)
        total += positive_rois
        print("{:5} {:5.2f}".format(positive_rois, positive_rois / ids.shape[1]))
    print("Average percent: {:.2f}".format(total / (limit * ids.shape[1])))

In [ ]:
np.max(idses[0][0] - idses[2][0])

In [ ]:
np.max(temp_g[0][1][0] - temp_g[3][1][0])

In [ ]:
temp_g[1][0][2]